# 三辺測量法

1. パッケージのimport

In [1145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import default_rng
from lmfit.models import SineModel
from scipy.optimize import curve_fit
from typing import Optional
import itertools
import sympy as sp
from sympy.plotting import plot3d
from point3d import Point3D
from point2d import Point2D
from antenna import Antenna
%matplotlib inline

In [1146]:
DATA_FILE_PATH = '..//data//results_100_60_50.csv'

2. 各定数の設定

In [1147]:
C = 2.998e8 #光の速度
F = 920.4e6 #周波数
wavelength = C / F * 100 #理論波長
PT = 32.5
G = 8.5
D0 = 100
N = 15
k = 0.02
beta = 2
alpha = 10 * np.log10(PT * G * G * pow(wavelength / (4 * np.pi * D0), 4) * 4 * np.pi * k /  pow(wavelength, 2))
print(wavelength)

32.57279443720122


3. 使用するタグの一覧

In [1148]:
TAG_TABLE = {
    'Tag1' : '2200 43CB 43A1 E48A 543D 29B9',
    'Tag2' : 'E200 470F 45D0 6023 A129 010E',
    'Tag3' : '9990 1912 9990 9990 9990 0137'
}
TARGET_TAG = 'Tag2'

In [1149]:
RSSI0_ANTENNA_NUMBER = 1
RSSI0_DATA_FILE_PATH = '..//data//RSSI0_GTag.csv'

単位RSSIの計算クラス

In [1150]:
class Rssi0:
    def __init__(self, csv_file:str, tag_table:dict, antenna_number:int, target_tag:str):
        self.df=pd.read_csv(csv_file)
        self.data_size = len(self.df)
        self.tag_table = tag_table
        self.antenna_number = antenna_number
        self.target_tag = target_tag
        self.df['Tag No.'] = self.df['Tag ID'].map(self.__tagTableInverseLookup)
        self.df = self.df[(self.df['Tag No.'] == self.target_tag) & (self.df['Antenna No.']==self.antenna_number)]
        self.df = self.df.drop(columns=['Tag ID']).set_index('Created At')

    def __tagTableInverseLookup(self, x:str):
        for k,v in self.tag_table.items():
            if x == v:
                return k

    def get_rssi0(self):
        return self.df[(self.df['Tag No.']==TARGET_TAG) & (self.df['Antenna No.']==self.antenna_number)]['RSSI'].mean()

4. 取得したデータのデータフレームを扱うクラス

In [1151]:
class RfidDf:
    def __init__(self, csv_file:str, tag_table:dict, antenna_list:list, target_tag:str):
        self.df=pd.read_csv(csv_file)
        self.data_size = len(self.df)
        self.tag_table = tag_table
        self.antenna_list = antenna_list
        self.target_tag = target_tag
        self.df['Distance'] = self.df['RSSI'].map(self.__culDistanceFromRSSI)
        self.df['Tag No.'] = self.df['Tag ID'].map(self.__tagTableInverseLookup)
        self.df = self.df[self.df['Tag No.'] == self.target_tag]
        self.df = self.df.drop(columns=['Tag ID']).set_index('Created At')

    def __culDistanceFromRSSI(self, rssi):
        rssi0 = Rssi0(
            csv_file=RSSI0_DATA_FILE_PATH,
            tag_table=self.tag_table,
            antenna_number=RSSI0_ANTENNA_NUMBER,
            target_tag=self.target_tag
        )
        return float(pow(10, (alpha - rssi)/(10 * beta)) * 100)

    def __culDistanceFromRSSIAndPhase(self, rssi:float, phase:float):
        pass

    def __tagTableInverseLookup(self, x:str):
        for k,v in self.tag_table.items():
            if x == v:
                return k

    def get_mean_of_target_tag(self):
        columns = ['RSSI', 'Distance', 'Phase', 'Antenna No.']
        data = np.empty((0,4))
        for j in self.antenna_list:
            rfiddf = self.df[(self.df['Tag No.'] == self.target_tag) & (self.df['Antenna No.'] == j.number)]
            data = np.append(
                data,
                [[
                    float(rfiddf['RSSI'].mean()), 
                    float(rfiddf['Distance'].mean()), 
                    float(rfiddf['Phase'].mean()),
                    rfiddf['Antenna No.'].mean()
                ]],
                axis=0
            )
        rfiddf2 = pd.DataFrame(
            columns=columns,
            data=data
        )
        return rfiddf2.set_index('Antenna No.')

    def getTagInfoList(self, info_type:str):
        return [[float(item) for item in self.df[self.df['Tag No.'] == f'Tag{i}'][info_type].tolist()] for i in range(1, len(self.tag_table)+1)]

5. 各種情報を可視化するクラス

In [1152]:
class RfidDfPlotter:
    def __init__(self, rfid_df:RfidDf):
        self.rfid_df = rfid_df
        self.infoTypeYticks = {
            'RSSI' : np.arange(-65, -39, 5),
            'Distance' : np.arange(0, 170, 16.8),
            'Phase' : np.arange(-6.5, 6.5, 0.5)
        }
        
    def rfiddfBoxplot(self, data_type:str):
        fig, ax = plt.subplots(figsize=(10,10))
        ax.boxplot(self.rfid_df.getTagInfoList(data_type))
        ax.set_xticklabels(list(self.rfid_df.tag_table.keys()))
        ax.set_yticks(self.infoTypeYticks[data_type])


In [1156]:
class Trilateration:
    def selectAntenna(self, antenna_list:list):
        #TODO アンテナセレクタの実装
        return antenna_list

    def __init__(self, target_tag:str, rfiddf:RfidDf, antenna_list:list) -> None:
        self.df = rfiddf
        self.target_tag = target_tag
        self.antenna_list = self.selectAntenna(antenna_list)

    def get_tag_location(self):
        d = []
        x, y, z = sp.symbols('x y z')
        for i in self.antenna_list:
            point = i.point
            l = i.culDistanceToTag(self.target_tag, self.df)
            d.append((point.x-x)**2 + (point.y-y)**2 + (point.z-z)**2 - l)
        result = sp.solve(d)
        results = []
        for i in result:
            results.append(
                Point3D(
                    x=i[x],
                    y=i[y],
                    z=i[z]
                )
            )
        return results

In [1157]:
antenna_list = [
    Antenna(point=Point3D(0,60,0),frequency=920.4,number=1),
    Antenna(point=Point3D(100,0,0),frequency=920.4,number=4),
    Antenna(point=Point3D(0,0,0), frequency=920.4, number=6)
]
rfiddf = RfidDf(
    csv_file=DATA_FILE_PATH,
    tag_table=TAG_TABLE,
    antenna_list=antenna_list,
    target_tag=TARGET_TAG
)
rfiddf.get_mean_of_target_tag().head()

,RSSI,Distance,Phase
Antenna No.,,,
1.0,-64.481818,87.533448,3.785307
4.0,-57.141509,37.403072,1.071587
6.0,-69.277778,150.188702,2.777528


In [1158]:
trilateration = Trilateration(target_tag=TARGET_TAG, rfiddf=rfiddf, antenna_list=antenna_list)
results = trilateration.get_tag_location()
for i in results:    
    print(f'x={i.x}, y={i.y}, z={i.z}')

x=50.5639281504094, y=30.5221271169330, z=-57.7764863248681*I
x=50.5639281504094, y=30.5221271169330, z=57.7764863248681*I
